In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [10]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. 1st attempt at optimization = drop more columns
application_df = application_df.drop(['EIN', 'NAME', 'APPLICATION_TYPE', 'CLASSIFICATION'], axis=1)
application_df

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,1,0,N,5000,1
1,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,Independent,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...
34294,Independent,ProductDev,Association,1,0,N,5000,0
34295,CompanySponsored,ProductDev,Association,1,0,N,5000,0
34296,CompanySponsored,Preservation,Association,1,0,N,5000,0
34297,Independent,ProductDev,Association,1,0,N,5000,1


In [11]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

application_df[application_cat].nunique()

AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True).drop(application_cat, 1)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
# Split our preprocessed data into our features and target arrays
X= application_df.drop(['IS_SUCCESSFUL'],1).values
y= application_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

/Users/emilymaynard/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                435       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 606
Trainable params: 606
Non-trainable params: 0
_________________________________________________________________


2022-08-21 14:19:18.426872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6257 - accuracy: 0.6720
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6015 - accuracy: 0.6960
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5997 - accuracy: 0.6989
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5992 - accuracy: 0.6992
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5989 - accuracy: 0.6992
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5992 - accuracy: 0.6999
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5988 - accuracy: 0.6998
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5986 - accuracy: 0.7000
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5984 - accuracy: 0.6997
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5987 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 0.7025
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5964 - accuracy: 0.7021
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5965 - accuracy: 0.7023
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 0.7025
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5962 - accuracy: 0.7025
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5964 - accuracy: 0.7023
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 0.7021
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 0.7025
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 0.7022
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5963 - accuracy: 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6003 - accuracy: 0.6960 - 539ms/epoch - 2ms/step
Loss: 0.600297749042511, Accuracy: 0.6959766745567322


## Second attempt, change columns and add nuerons

In [21]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [22]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. 1st attempt at optimization = drop more columns
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [23]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df['APPLICATION_TYPE'].value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type[app_type < 528].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [25]:
# Look at CLASSIFICATION value counts for binning
class_counts= application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [26]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts<1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [27]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

application_df[application_cat].nunique()

APPLICATION_TYPE    9
AFFILIATION         6
CLASSIFICATION      6
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
dtype: int64

In [28]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True).drop(application_cat, 1)
application_df.head()

/Users/emilymaynard/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Split our preprocessed data into our features and target arrays
X= application_df.drop(['IS_SUCCESSFUL'],1).values
y= application_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

/Users/emilymaynard/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
from tensorflow.keras.layers import LeakyReLU, Dense
leaky_relu = LeakyReLU(alpha=0.01)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Add another layer to see if it increaases accuracy. add more nuerons
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="softmax"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="leaky_relu")) 

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 32)                1312      
                                                                 
 dense_17 (Dense)            (None, 12)                396       
                                                                 
 dense_18 (Dense)            (None, 5)                 65        
                                                                 
 dense_19 (Dense)            (None, 1)                 6         
                                                                 
Total params: 1,779
Trainable params: 1,779
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [43]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [44]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
  1/804 [..............................] - ETA: 8:31 - loss: 0.6849 - accuracy: 0.5938
Epoch 1: saving model to checkpoints/weights.01.hdf5
  9/804 [..............................] - ETA: 4s - loss: 0.6901 - accuracy: 0.5139  
Epoch 1: saving model to checkpoints/weights.01.hdf5
 10/804 [..............................] - ETA: 10s - loss: 0.6908 - accuracy: 0.5031
Epoch 1: saving model to checkpoints/weights.01.hdf5
 15/804 [..............................] - ETA: 13s - loss: 0.6936 - accuracy: 0.4771
Epoch 1: saving model to checkpoints/weights.01.hdf5
 20/804 [..............................] - ETA: 13s - loss: 0.6961 - accuracy: 0.4563
Epoch 1: saving model to checkpoints/weights.01.hdf5
 25/804 [..............................] - ETA: 13s - loss: 0.6956 - accuracy: 0.4613
Epoch 1: saving model to checkpoints/weights.01.hdf5
 30/804 [>.............................] - ETA: 12s - loss: 0.6952 - accuracy: 0.4615
Epoch 1: saving model to checkpoints/weights.01.hdf5
 35/804 [>...


Epoch 1: saving model to checkpoints/weights.01.hdf5
320/804 [==========>...................] - ETA: 5s - loss: 0.6406 - accuracy: 0.6637
Epoch 1: saving model to checkpoints/weights.01.hdf5
325/804 [===========>..................] - ETA: 5s - loss: 0.6394 - accuracy: 0.6654
Epoch 1: saving model to checkpoints/weights.01.hdf5
330/804 [===========>..................] - ETA: 5s - loss: 0.6388 - accuracy: 0.6660
Epoch 1: saving model to checkpoints/weights.01.hdf5
335/804 [===========>..................] - ETA: 5s - loss: 0.6381 - accuracy: 0.6667
Epoch 1: saving model to checkpoints/weights.01.hdf5
342/804 [===========>..................] - ETA: 5s - loss: 0.6373 - accuracy: 0.6678
Epoch 1: saving model to checkpoints/weights.01.hdf5

Epoch 1: saving model to checkpoints/weights.01.hdf5
350/804 [============>.................] - ETA: 5s - loss: 0.6364 - accuracy: 0.6687
Epoch 1: saving model to checkpoints/weights.01.hdf5
355/804 [============>.................] - ETA: 5s - loss: 0.634

640/804 [======================>.......] - ETA: 1s - loss: 0.6076 - accuracy: 0.6960
Epoch 1: saving model to checkpoints/weights.01.hdf5
645/804 [=======================>......] - ETA: 1s - loss: 0.6075 - accuracy: 0.6961
Epoch 1: saving model to checkpoints/weights.01.hdf5
650/804 [=======================>......] - ETA: 1s - loss: 0.6074 - accuracy: 0.6962
Epoch 1: saving model to checkpoints/weights.01.hdf5
656/804 [=======================>......] - ETA: 1s - loss: 0.6072 - accuracy: 0.6965
Epoch 1: saving model to checkpoints/weights.01.hdf5
660/804 [=======================>......] - ETA: 1s - loss: 0.6070 - accuracy: 0.6966
Epoch 1: saving model to checkpoints/weights.01.hdf5
665/804 [=======================>......] - ETA: 1s - loss: 0.6065 - accuracy: 0.6970
Epoch 1: saving model to checkpoints/weights.01.hdf5
670/804 [========================>.....] - ETA: 1s - loss: 0.6059 - accuracy: 0.6976
Epoch 1: saving model to checkpoints/weights.01.hdf5
675/804 [========================>

191/804 [======>.......................] - ETA: 4s - loss: 0.5608 - accuracy: 0.7273
Epoch 2: saving model to checkpoints/weights.02.hdf5
198/804 [======>.......................] - ETA: 4s - loss: 0.5607 - accuracy: 0.7273
Epoch 2: saving model to checkpoints/weights.02.hdf5
205/804 [======>.......................] - ETA: 4s - loss: 0.5612 - accuracy: 0.7262
Epoch 2: saving model to checkpoints/weights.02.hdf5
210/804 [======>.......................] - ETA: 4s - loss: 0.5604 - accuracy: 0.7263
Epoch 2: saving model to checkpoints/weights.02.hdf5

Epoch 2: saving model to checkpoints/weights.02.hdf5
216/804 [=======>......................] - ETA: 4s - loss: 0.5599 - accuracy: 0.7271
Epoch 2: saving model to checkpoints/weights.02.hdf5
221/804 [=======>......................] - ETA: 4s - loss: 0.5600 - accuracy: 0.7272
Epoch 2: saving model to checkpoints/weights.02.hdf5
226/804 [=======>......................] - ETA: 4s - loss: 0.5596 - accuracy: 0.7276
Epoch 2: saving model to checkpoi

537/804 [===================>..........] - ETA: 2s - loss: 0.5582 - accuracy: 0.7276
Epoch 2: saving model to checkpoints/weights.02.hdf5
544/804 [===================>..........] - ETA: 2s - loss: 0.5583 - accuracy: 0.7272
Epoch 2: saving model to checkpoints/weights.02.hdf5

Epoch 2: saving model to checkpoints/weights.02.hdf5
551/804 [===================>..........] - ETA: 2s - loss: 0.5583 - accuracy: 0.7271
Epoch 2: saving model to checkpoints/weights.02.hdf5
560/804 [===================>..........] - ETA: 2s - loss: 0.5589 - accuracy: 0.7263
Epoch 2: saving model to checkpoints/weights.02.hdf5

Epoch 2: saving model to checkpoints/weights.02.hdf5
566/804 [====================>.........] - ETA: 2s - loss: 0.5586 - accuracy: 0.7264
Epoch 2: saving model to checkpoints/weights.02.hdf5
573/804 [====================>.........] - ETA: 2s - loss: 0.5585 - accuracy: 0.7266
Epoch 2: saving model to checkpoints/weights.02.hdf5

Epoch 2: saving model to checkpoints/weights.02.hdf5
581/804 [=

...

 43/804 [>.............................] - ETA: 6s - loss: 0.5246 - accuracy: 0.7464
Epoch 99: saving model to checkpoints/weights.99.hdf5
 52/804 [>.............................] - ETA: 6s - loss: 0.5237 - accuracy: 0.7488
Epoch 99: saving model to checkpoints/weights.99.hdf5

Epoch 99: saving model to checkpoints/weights.99.hdf5
 58/804 [=>............................] - ETA: 6s - loss: 0.5200 - accuracy: 0.7500
Epoch 99: saving model to checkpoints/weights.99.hdf5
 63/804 [=>............................] - ETA: 6s - loss: 0.5287 - accuracy: 0.7450
Epoch 99: saving model to checkpoints/weights.99.hdf5
 69/804 [=>............................] - ETA: 6s - loss: 0.5303 - accuracy: 0.7409
Epoch 99: saving model to checkpoints/weights.99.hdf5
 77/804 [=>............................] - ETA: 6s - loss: 0.5337 - accuracy: 0.7366
Epoch 99: saving model to checkpoints/weights.99.hdf5

Epoch 99: saving model to checkpoints/weights.99.hdf5
 83/804 [==>...........................] - ETA: 6s - los

390/804 [=============>................] - ETA: 3s - loss: 0.5360 - accuracy: 0.7395
Epoch 99: saving model to checkpoints/weights.99.hdf5

Epoch 99: saving model to checkpoints/weights.99.hdf5
398/804 [=============>................] - ETA: 3s - loss: 0.5370 - accuracy: 0.7382
Epoch 99: saving model to checkpoints/weights.99.hdf5
406/804 [==============>...............] - ETA: 3s - loss: 0.5364 - accuracy: 0.7387
Epoch 99: saving model to checkpoints/weights.99.hdf5

Epoch 99: saving model to checkpoints/weights.99.hdf5
413/804 [==============>...............] - ETA: 3s - loss: 0.5366 - accuracy: 0.7384
Epoch 99: saving model to checkpoints/weights.99.hdf5
422/804 [==============>...............] - ETA: 3s - loss: 0.5359 - accuracy: 0.7388
Epoch 99: saving model to checkpoints/weights.99.hdf5

Epoch 99: saving model to checkpoints/weights.99.hdf5
428/804 [==============>...............] - ETA: 3s - loss: 0.5368 - accuracy: 0.7379
Epoch 99: saving model to checkpoints/weights.99.hdf5
4

738/804 [==========================>...] - ETA: 0s - loss: 0.5380 - accuracy: 0.7371
Epoch 99: saving model to checkpoints/weights.99.hdf5
746/804 [==========================>...] - ETA: 0s - loss: 0.5382 - accuracy: 0.7374
Epoch 99: saving model to checkpoints/weights.99.hdf5
751/804 [===========================>..] - ETA: 0s - loss: 0.5377 - accuracy: 0.7379
Epoch 99: saving model to checkpoints/weights.99.hdf5

Epoch 99: saving model to checkpoints/weights.99.hdf5
758/804 [===========================>..] - ETA: 0s - loss: 0.5375 - accuracy: 0.7383
Epoch 99: saving model to checkpoints/weights.99.hdf5
763/804 [===========================>..] - ETA: 0s - loss: 0.5380 - accuracy: 0.7379
Epoch 99: saving model to checkpoints/weights.99.hdf5
770/804 [===========================>..] - ETA: 0s - loss: 0.5376 - accuracy: 0.7381
Epoch 99: saving model to checkpoints/weights.99.hdf5
773/804 [===========================>..] - ETA: 0s - loss: 0.5377 - accuracy: 0.7381
Epoch 99: saving model to 

246/804 [========>.....................] - ETA: 5s - loss: 0.5351 - accuracy: 0.7398
Epoch 100: saving model to checkpoints/weights.100.hdf5
250/804 [========>.....................] - ETA: 5s - loss: 0.5351 - accuracy: 0.7393
Epoch 100: saving model to checkpoints/weights.100.hdf5
254/804 [========>.....................] - ETA: 5s - loss: 0.5341 - accuracy: 0.7399
Epoch 100: saving model to checkpoints/weights.100.hdf5
259/804 [========>.....................] - ETA: 5s - loss: 0.5341 - accuracy: 0.7397
Epoch 100: saving model to checkpoints/weights.100.hdf5
264/804 [========>.....................] - ETA: 5s - loss: 0.5349 - accuracy: 0.7398
Epoch 100: saving model to checkpoints/weights.100.hdf5
269/804 [=========>....................] - ETA: 5s - loss: 0.5350 - accuracy: 0.7401
Epoch 100: saving model to checkpoints/weights.100.hdf5
274/804 [=========>....................] - ETA: 5s - loss: 0.5351 - accuracy: 0.7401
Epoch 100: saving model to checkpoints/weights.100.hdf5
279/804 [====

554/804 [===================>..........] - ETA: 2s - loss: 0.5389 - accuracy: 0.7369
Epoch 100: saving model to checkpoints/weights.100.hdf5
561/804 [===================>..........] - ETA: 2s - loss: 0.5388 - accuracy: 0.7370
Epoch 100: saving model to checkpoints/weights.100.hdf5

Epoch 100: saving model to checkpoints/weights.100.hdf5
569/804 [====================>.........] - ETA: 2s - loss: 0.5381 - accuracy: 0.7378
Epoch 100: saving model to checkpoints/weights.100.hdf5
578/804 [====================>.........] - ETA: 2s - loss: 0.5375 - accuracy: 0.7382
Epoch 100: saving model to checkpoints/weights.100.hdf5

Epoch 100: saving model to checkpoints/weights.100.hdf5
584/804 [====================>.........] - ETA: 2s - loss: 0.5368 - accuracy: 0.7386
Epoch 100: saving model to checkpoints/weights.100.hdf5
593/804 [=====================>........] - ETA: 2s - loss: 0.5364 - accuracy: 0.7389
Epoch 100: saving model to checkpoints/weights.100.hdf5

Epoch 100: saving model to checkpoints/

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5566 - accuracy: 0.7279 - 806ms/epoch - 3ms/step
Loss: 0.556611955165863, Accuracy: 0.7279300093650818


## Attempt 3
### use auto optimizer

In [56]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [57]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [58]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 508 Complete [00h 06m 04s]
val_accuracy: 0.7314285635948181

Best val_accuracy So Far: 0.7351603507995605
Total elapsed time: 04h 50m 04s
INFO:tensorflow:Oracle triggered exit


In [59]:
 # Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 23,
 'num_layers': 4,
 'units_0': 3,
 'units_1': 9,
 'units_2': 1,
 'units_3': 5,
 'units_4': 3,
 'units_5': 3,
 'tuner/epochs': 200,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [60]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7352 - 632ms/epoch - 2ms/step
Loss: 0.5545336604118347, Accuracy: 0.7351603507995605


In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Add another layer to see if it increaases accuracy. add more nuerons
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 23
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 5
hidden_nodes_layer4 = 3


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")) 
#add hidden layer 4
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu")) 

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model # 200 epochs was also tried with similar results
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 23)                943       
                                                                 
 dense_15 (Dense)            (None, 9)                 216       
                                                                 
 dense_16 (Dense)            (None, 5)                 50        
                                                                 
 dense_17 (Dense)            (None, 3)                 18        
                                                                 
 dense_18 (Dense)            (None, 1)                 4         
                                                                 
Total params: 1,231
Trainable params: 1,231
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] 

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5557 - accuracy: 0.7286 - 502ms/epoch - 2ms/step
Loss: 0.5556609034538269, Accuracy: 0.7286297082901001


In [66]:
nn.save("AlphabetSoupCharity_Optimization.h5")